In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from transformers import T5Tokenizer, TFAutoModelForSeq2SeqLM
import pandas as pd
import ast

In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Project 2/Copy of cleaned_data.csv')
data.head()

,headline,title,text,lemmatized headline,lemmatized text,input_ids,attention_mask,labels
0,keep related supplies area make effort clean d...,How to Be an Organized Artist1,photographer keep necessary lens cords batteri...,keep related supply area make effort clean ded...,photographer keep necessary lens cord battery ...,"[9050, 453, 1316, 6888, 7496, 3322, 16429, 356...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[453, 1341, 1899, 616, 143, 1941, 1349, 2425, ..."
1,create sketch neopoprealist manner future mura...,How to Create a Neopoprealist Art Work,see image drawing develops step step however i...,create sketch neopoprealist manner future mura...,see image drawing develops step step however i...,"[217, 1023, 5364, 1344, 7, 1147, 1147, 983, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[482, 13278, 3, 29, 15, 32, 9791, 6644, 343, 3..."
2,get bachelor degree enroll studio based progra...,How to Be a Visual Effects Artist1,possible become vfx artist without college deg...,get bachelor degree enroll studio based progra...,possible become vfx artist without college deg...,"[487, 582, 3, 208, 89, 226, 2377, 406, 1900, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[129, 16428, 1952, 17990, 3100, 3, 390, 478, 2..."
3,start experience interest art understand diffe...,How to Become an Art Investor,best art investors research pieces art buy som...,start experience interest art understand diffe...,best art investor research piece art buy someo...,"[200, 768, 12024, 585, 1466, 768, 805, 841, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[456, 351, 1046, 768, 734, 1750, 768, 12501, 7..."
4,keep reference materials sketches articles pho...,How to Be an Organized Artist2,start planning project work likely gathering s...,keep reference material sketch article photo e...,start planning project work likely gathering s...,"[456, 1459, 516, 161, 952, 7241, 7346, 3773, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[453, 2848, 1037, 13278, 1108, 1202, 672, 80, ..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209110 entries, 0 to 209109
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   headline             209110 non-null  object
 1   title                209110 non-null  object
 2   text                 209110 non-null  object
 3   lemmatized headline  209110 non-null  object
 4   lemmatized text      209110 non-null  object
 5   input_ids            209110 non-null  object
 6   attention_mask       209110 non-null  object
 7   labels               209110 non-null  object
dtypes: object(8)
memory usage: 12.8+ MB


In [ ]:
df_sampled = data.sample(n=100000, random_state=42)  # Selects 50,000 random rows
df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 151785 to 118706
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   headline             100000 non-null  object
 1   title                100000 non-null  object
 2   text                 100000 non-null  object
 3   lemmatized headline  100000 non-null  object
 4   lemmatized text      100000 non-null  object
 5   input_ids            100000 non-null  object
 6   attention_mask       100000 non-null  object
 7   labels               100000 non-null  object
dtypes: object(8)
memory usage: 6.9+ MB


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))  # Should show GPU details


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Convert string representations of lists back to lists for sampled data
df_sampled["input_ids"] = df_sampled["input_ids"].apply(ast.literal_eval)
df_sampled["attention_mask"] = df_sampled["attention_mask"].apply(ast.literal_eval)
df_sampled["labels"] = df_sampled["labels"].apply(ast.literal_eval)

def encode_data(sample):
    return {
        "input_ids": tf.convert_to_tensor(sample["input_ids"], dtype=tf.int32),
        "attention_mask": tf.convert_to_tensor(sample["attention_mask"], dtype=tf.int32),
        "labels": tf.convert_to_tensor(sample["labels"], dtype=tf.int32),
    }

# Create dataset from sampled data
dataset = tf.data.Dataset.from_generator(
    lambda: (encode_data(row) for _, row in df_sampled.iterrows()),
    output_signature={
        "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "attention_mask": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "labels": tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
)

# Batch and shuffle
BATCH_SIZE = 8  # Adjust as needed
dataset = dataset.shuffle(5000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
model_name = "t5-small"  # Choose "t5-base" for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn)

In [ ]:
model.fit(dataset, epochs=3)


Epoch 1/3
12500/12500 [==============================] - 5314s 421ms/step - loss: 1.5110
Epoch 2/3
12500/12500 [==============================] - 5271s 421ms/step - loss: 1.3268
Epoch 3/3
12500/12500 [==============================] - 5274s 422ms/step - loss: 1.2979


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Project 2/t5_tf_text_summarization")
tokenizer.save_pretrained("/content/drive/MyDrive/Project 2/t5_tf_text_summarization")

('/content/drive/MyDrive/Project 2/t5_tf_text_summarization/tokenizer_config.json',
 '/content/drive/MyDrive/Project 2/t5_tf_text_summarization/special_tokens_map.json',
 '/content/drive/MyDrive/Project 2/t5_tf_text_summarization/spiece.model',
 '/content/drive/MyDrive/Project 2/t5_tf_text_summarization/added_tokens.json')

In [ ]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="tf", max_length=512, truncation=True)
    summary_ids = model.generate(**inputs, max_length=150, num_beams=5)
    return tokenizer.decode(summary_ids[0].numpy(), skip_special_tokens=True)

test_text = "Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. It is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. In the healthcare sector, AI-driven tools assist doctors in diagnosing diseases, predicting patient outcomes, and even performing robotic surgeries. Financial institutions leverage AI for fraud detection, algorithmic trading, and customer service chatbots. Meanwhile, in the entertainment industry, AI personalizes content recommendations, enhances gaming experiences, and even assists in scriptwriting.Despite these advancements, AI also presents challenges. Concerns over data privacy and security have been raised as AI systems collect and analyze massive amounts of user data. Additionally, job displacement due to automation is a growing concern, with many fearing that AI will replace human workers in various fields. Policymakers and researchers emphasize the importance of developing ethical AI frameworks to ensure responsible deployment and mitigate potential risks.As AI continues to evolve, it is crucial to balance its benefits with ethical considerations. With the right policies and safeguards in place, AI has the potential to drive innovation while addressing societal concerns."
print("Summary:", summarize(test_text))

Summary: . AI is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. AI is reshaping industries by automating processes, enhancing decision-making, and improving efficiency. In the healthcare sector, AI-driven tools assist doctors in diagnosing diseases, predicting patient outcomes, and even performing robotic surgeries. In the entertainment industry, AI personalizes content recommendations, enhances gaming experiences, and even assists in scriptwriting.
